<a href="https://colab.research.google.com/gist/alejandro-ao/124673ecfc2e860cbb592e9cfeeae256/deep-research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Research Agent with Open Models

## Setup

In [ ]:
%pip install -Uq "smolagents[mcp,litellm,openai]" huggingface_hub

In [ ]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["FIRECRAWL_API_KEY"] = getpass.getpass("Firecrawl API Key:")
os.environ["HF_TOKEN"] = getpass.getpass("Hugging Face Token:")

OpenAI API Key:··········
Firecrawl API Key:··········
Hugging Face Token:··········


## Generate research plan

First, we will have the LLM generate a research plan that will serve as anchor for all our sub agents.


In [ ]:
PLANNER_SYSTEM_INSTRUCTIONS= """
You will be given a research task by a user. Your job is to produce a set of
instructions for a researcher that will complete the task. Do NOT complete the
task yourself, just provide instructions on how to complete it.

GUIDELINES:
1. Maximize specificity and detail. Include all known user preferences and
   explicitly list key attributes or dimensions to consider.
2. If essential attributes are missing, explicitly state that they are open-ended.
3. Avoid unwarranted assumptions. Treat unspecified dimensions as flexible.
4. Use the first person (from the user's perspective).
5. When helpful, explicitly ask the researcher to include tables.
6. Include the expected output format (e.g. structured report with headers).
7. Preserve the input language unless the user explicitly asks otherwise.
8. Sources: prefer primary / official / original sources.
"""


from huggingface_hub import InferenceClient

def generate_research_plan(user_query: str) -> str:
    MODEL_ID = "moonshotai/Kimi-K2-Thinking"
    PROVIDER = "together"

    print("Generating the research plan for the query: ", user_query)
    print("MODEL: ", MODEL_ID)
    print("PROVIDER: ", PROVIDER)

    planner_client = InferenceClient(
        api_key=os.environ["HF_TOKEN"],
        provider=PROVIDER,
        bill_to="huggingface",
    )
    completion = planner_client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": PLANNER_SYSTEM_INSTRUCTIONS},
            {"role": "user", "content": user_query},
        ],
    )

    research_plan = completion.choices[0].message.content

    print("\033[93mGenerated Research Plan\033[0m")
    print(f"\033[93m{research_plan}\033[0m")

    return research_plan

research_plan = generate_research_plan("research about the climate in northern france")

Generating the research plan for the query:  research about the climate in northern france
MODEL:  moonshotai/Kimi-K2-Thinking
PROVIDER:  together
Generated Research Plan
I need a comprehensive research report on the climate of northern France. Please follow these detailed instructions:

---

### **1. Geographic and Temporal Scope (CLARIFICATION REQUIRED)**

First, confirm the exact geographic boundaries I want to study. Since "northern France" is ambiguous, **explicitly state your interpretation** and provide data for both possible definitions:
- **Definition A**: Administrative regions (Hauts-de-France, Normandy, Île-de-France, and parts of Brittany)
- **Definition B**: Geographic threshold (all departments north of the Loire River)

**Time periods to cover** (these are open-ended; gather what's available):
- **Historical baseline**: 1961-1990 or 1981-2010 climate normals
- **Recent trends**: 2000-2023 data
- **Future projections**: Include at least two IPCC scenarios (e.g., SSP2-4.5

## Split into subtasks

Next, we will split the research plan into multiple research subtasks that will be run by our subagents agents.

In [ ]:
import json
from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

TASK_SPLITTER_SYSTEM_INSTRUCTIONS = """
You will be given a set of research instructions (a research plan).
Your job is to break this plan into a set of coherent, non-overlapping
subtasks that can be researched independently by separate agents.

Requirements:
- 3 to 8 subtasks is usually a good range. Use your judgment.
- Each subtask should have:
  - an 'id' (short string),
  - a 'title' (short descriptive title),
  - a 'description' (clear, detailed instructions for the sub-agent).
- Subtasks should collectively cover the full scope of the original plan
  without unnecessary duplication.
- Prefer grouping by dimensions: time periods, regions, actors, themes,
  causal mechanisms, etc., depending on the topic.
- Each description should be very clear and detailed about everything that
  the agent needs to research to cover that topic.
- Do not include a final task that will put everything together.
  This will be done later in another step.

Output format:
Return ONLY valid JSON with this schema:

{
  "subtasks": [
    {
      "id": "string",
      "title": "string",
      "description": "string"
    }
  ]
}
"""

class Subtask(BaseModel):
    id: str = Field(
        ...,
        description="Short identifier for the subtask (e.g. 'A', 'history', 'drivers').",
    )
    title: str = Field(
        ...,
        description="Short descriptive title of the subtask.",
    )
    description: str = Field(
        ...,
        description="Clear, detailed instructions for the sub-agent that will research this subtask.",
    )

class SubtaskList(BaseModel):
    subtasks: List[Subtask] = Field(
        ...,
        description="List of subtasks that together cover the whole research plan.",
    )

TASK_SPLITTER_JSON_SCHEMA = {
    "name": "subtaskList",
    "schema": SubtaskList.model_json_schema(),
    "strict": True,
}


def split_into_subtasks(research_plan: str):

    MODEL_ID = "moonshotai/Kimi-K2-Thinking"
    PROVIDER = "nebius"

    print("Splitting the research plan into subtasks...")
    print("MODEL: ", MODEL_ID)
    print("PROVIDER: ", PROVIDER)

    client = InferenceClient(
      api_key=os.environ["HF_TOKEN"],
      provider=PROVIDER,
      bill_to="huggingface",
    )

    completion = client.chat_completion(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": TASK_SPLITTER_SYSTEM_INSTRUCTIONS},
            {"role": "user", "content": research_plan},
        ],
        response_format={
            "type": "json_schema",
            "json_schema": TASK_SPLITTER_JSON_SCHEMA,
        },
    )

    message = completion.choices[0].message

    print(message)

    subtasks = json.loads(message.content)['subtasks']

    print("\033[93mGenerated The Following Subtasks\033[0m")
    for task in subtasks:
      print(f"\033[93m{task['title']}\033[0m")
      pprint(f"\033[93m{task['description']}\033[0m")
      print()

    return subtasks

subtasks = split_into_subtasks(research_plan)



Splitting the research plan into subtasks...
MODEL:  moonshotai/Kimi-K2-Thinking
PROVIDER:  nebius
ChatCompletionOutputMessage(role='assistant', content=' {"subtasks":[{"id":"geographic_baseline","title":"Geographic Scope Definition and Baseline Climate Normals (1991-2020)","description":"Define \'northern France\' as the administrative regions of Hauts-de-France and Normandy, plus the northern portions of Grand Est and Brittany. Establish five representative reference cities: Lille (inland urban), Calais (coastal Channel), Rouen (Seine valley), Le Havre (coastal Atlantic), and Amiens (inland plain). For each location, gather comprehensive climate normals for the 1991-2020 WMO standard period from Météo-France Climate Atlas and ECMWF ERA5 reanalysis. Collect all core variables: mean monthly/annual temperatures and daily highs/lows; absolute temperature records with dates; frost day counts; growing season length; sea surface temperatures for coastal cities; monthly/annual precipitation 

## Create subagents + coordinator

In this step, we’ll create a tool that spins up a dedicated sub-agent for each subtask. This tool will be handed to the Coordinator agent, which will invoke it whenever a new subtask needs to be processed. Each sub-agent will perform thorough research on its assigned subtask and return its findings once completed. The Coordinator will then aggregate all sub-agent outputs into a comprehensive deep-research report.

In [ ]:
SUBAGENT_PROMPT_TEMPLATE = """
You are a specialized research sub-agent.

Global user query:
{user_query}

Overall research plan:
{research_plan}

Your specific subtask (ID: {subtask_id}, Title: {subtask_title}) is:

\"\"\"{subtask_description}\"\"\"

Instructions:
- Focus ONLY on this subtask, but keep the global query in mind for context.
- Use the available tools to search for up-to-date, high-quality sources.
- Prioritize primary and official sources when possible.
- Be explicit about uncertainties, disagreements in the literature, and gaps.
- Return your results as a MARKDOWN report with this structure:

# [Subtask ID] [Subtask Title]

## Summary
Short overview of the main findings.

## Detailed Analysis
Well-structured explanation with subsections as needed.

## Key Points
- Bullet point
- Bullet point

## Sources
- [Title](url) - short comment on why this source is relevant

Now perform the research and return ONLY the markdown report.
"""

COORDINATOR_PROMPT_TEMPLATE = """
You are the LEAD RESEARCH COORDINATOR AGENT.

The user has asked:
\"\"\"{user_query}\"\"\"

A detailed research plan has already been created:

\"\"\"{research_plan}\"\"\"

This plan has been split into the following subtasks (JSON):

```json
{subtasks_json}
```
Each element has the shape:
{{
“id”: “timeframe_confirmation”,
“title”: “Confirm Research Scope Parameters”,
“description”: “Analyze the scope parameters…”
}}

You have access to a tool called:
• initialize_subagent(subtask_id: str, subtask_title: str, subtask_description: str)

Your job:
1. For EACH subtask in the JSON array, call initialize_subagent exactly once
with:
• subtask_id       = subtask[“id”]
• subtask_title    = subtask[“title”]
• subtask_description = subtask[“description”]
2. Wait for all sub-agent reports to come back. Each tool call returns a
markdown report for that subtask.
3. After you have results for ALL subtasks, synthesize them into a SINGLE,
coherent, deeply researched report addressing the original user query
("{user_query}").

Final report requirements:
• Integrate all sub-agent findings; avoid redundancy.
• Make the structure clear with headings and subheadings.
• Highlight:
• key drivers and mechanisms of insecurity,
• historical and temporal evolution,
• geographic and thematic patterns,
• state capacity, public perception, and socioeconomic correlates,
• open questions and uncertainties.
• Include final sections:
• Open Questions and Further Research
• Bibliography / Sources: merge and deduplicate the key sources from all sub-agents.

Important:
• DO NOT expose internal tool-call mechanics to the user.
• Your final answer to the user should be a polished markdown report.
"""


In [ ]:
from smolagents import InferenceClientModel, MCPClient, tool, ToolCallingAgent
import os

FIRECRAWL_API_KEY = os.environ["FIRECRAWL_API_KEY"]
MCP_URL = f"https://mcp.firecrawl.dev/{FIRECRAWL_API_KEY}/v2/mcp"

COORDINATOR_MODEL_ID = "MiniMaxAI/MiniMax-M1-80k"
SUBAGENT_MODEL_ID = "MiniMaxAI/MiniMax-M1-80k"
COORDINATOR_MODEL_PROVIDER = "novita"
SUBAGENT_MODEL_PROVIDER = "novita"

def run_deep_research(user_query: str) -> str:
    print("Running the deep research...")

    # 1) Generate research plan
    research_plan = generate_research_plan(user_query)

    # 2) Split into explicit subtasks
    subtasks = split_into_subtasks(research_plan)

    print("Initializing Coordinator")
    print("Coordinator Model: ", COORDINATOR_MODEL_ID)
    print("Subagent Model: ", SUBAGENT_MODEL_ID)

    coordinator_model = InferenceClientModel(
        model_id=COORDINATOR_MODEL_ID,
        api_key=os.environ["HF_TOKEN"],
        provider=COORDINATOR_MODEL_PROVIDER,
        bill_to="huggingface"
        )
    subagent_model = InferenceClientModel(
        model_id=SUBAGENT_MODEL_ID,
        api_key=os.environ["HF_TOKEN"],
        provider=SUBAGENT_MODEL_PROVIDER,
        bill_to="huggingface"
        )

    with MCPClient({"url": MCP_URL, "transport": "streamable-http"}) as mcp_tools:
        @tool
        def initialize_subagent(subtask_id: str, subtask_title: str, subtask_description: str) -> str:
            """
           Spawn a dedicated research sub-agent for a single subtask.

            Args:
                subtask_id (str): The unique identifier for the subtask.
                subtask_title (str): The descriptive title of the subtask.
                subtask_description (str): Detailed instructions for the sub-agent to perform the subtask.

            The sub-agent:
            - Has access to the Firecrawl MCP tools.
            - Must perform deep research ONLY on this subtask.
            - Returns a structured markdown report with:
              - a clear heading identifying the subtask,
              - a narrative explanation,
              - bullet-point key findings,
              - explicit citations / links to sources.
            """
            print(f"Initializing Subagent for task {subtask_id}...")

            subagent = ToolCallingAgent(
                tools=mcp_tools,                # Firecrawl MCP toolkit
                model=subagent_model,
                add_base_tools=False,
                name=f"subagent_{subtask_id}",
            )
            subagent_prompt = SUBAGENT_PROMPT_TEMPLATE.format(
                user_query=user_query,
                research_plan=research_plan,
                subtask_id=subtask_id,
                subtask_title=subtask_title,
                subtask_description=subtask_description,
            )

            return subagent.run(subagent_prompt)

        coordinator = ToolCallingAgent(
            tools=[initialize_subagent],
            model=coordinator_model,
            add_base_tools=False,
            name="coordinator_agent"
        )

        subtasks_json = json.dumps(subtasks, indent=2, ensure_ascii=False)

        coordinator_prompt = COORDINATOR_PROMPT_TEMPLATE.format(
            user_query=user_query,
            research_plan=research_plan,
            subtasks_json=subtasks_json,
        )

        final_report = coordinator.run(coordinator_prompt)

        return final_report







## Final result

This is the result of running the entire Deep Research Pipeline

In [ ]:
result = run_deep_research("research the climate in northern france")

Running the deep research...
Generating the research plan for the query:  research the climate in northern france
MODEL:  moonshotai/Kimi-K2-Thinking
PROVIDER:  together
Generated Research Plan
**Research Brief: Climate in Northern France**

I need a comprehensive research report on the climate characteristics of northern France. Since some parameters are undefined, please either confirm these choices with me or proceed using the standard recommendations below and explicitly state your assumptions in the report.

---

### **1. GEOGRAPHIC SCOPE (To Be Defined)**
Clarify the spatial boundaries before proceeding:
- **Preferred Definition**: Use administrative regions of Hauts-de-France, Normandy, and the northern portions of Grand Est (Ardennes, Meuse, Meurthe-et-Moselle) and Île-de-France.
- **Key Distinction**: Separate analysis for:
  - Coastal zones (Channel/North Sea: from Dunkirk to Le Havre)
  - Inland areas (Artois, Flanders, Picardy plateaus)
  - Urban heat island effects (Lille,

/tmp/ipython-input-1829780787.py:38: FutureWarning: Parameter 'structured_output' was not specified. Currently it defaults to False, but in version 1.25, the default will change to True. To suppress this warning, explicitly set structured_output=True (new behavior) or structured_output=False (legacy behavior). See documentation at https://huggingface.co/docs/smolagents/tutorials/tools#structured-output-and-output-schema-support for more details.
  with MCPClient({"url": MCP_URL, "transport": "streamable-http"}) as mcp_tools:


╭────────────────────────────────────────── New run - coordinator_agent ──────────────────────────────────────────╮
│                                                                                                                 │
│ You are the LEAD RESEARCH COORDINATOR AGENT.                                                                    │
│                                                                                                                 │
│ The user has asked:                                                                                             │
│ """research the climate in northern france"""                                                                   │
│                                                                                                                 │
│ A detailed research plan has already been created:                                                              │
│                                                                                                                 │
│ """**Research Brief: Climate in Northern France**                                                               │
│                                                                                                                 │
│ I need a comprehensive research report on the climate characteristics of northern France. Since some parameters │
│ are undefined, please either confirm these choices with me or proceed using the standard recommendations below  │
│ and explicitly state your assumptions in the report.                                                            │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ### **1. GEOGRAPHIC SCOPE (To Be Defined)**                                                                     │
│ Clarify the spatial boundaries before proceeding:                                                               │
│ - **Preferred Definition**: Use administrative regions of Hauts-de-France, Normandy, and the northern portions  │
│ of Grand Est (Ardennes, Meuse, Meurthe-et-Moselle) and Île-de-France.                                           │
│ - **Key Distinction**: Separate analysis for:                                                                   │
│   - Coastal zones (Channel/North Sea: from Dunkirk to Le Havre)                                                 │
│   - Inland areas (Artois, Flanders, Picardy plateaus)                                                           │
│   - Urban heat island effects (Lille, Rouen, Strasbourg, Paris comparative data)                                │
│ - If I don't respond, **proceed with the four-region definition above** and create sub-sections for coastal vs. │
│ inland climates.                                                                                                │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ### **2. TIME PERIOD COVERAGE (To Be Defined)**                                                                 │
│ Specify the temporal focus:                                                                                     │
│ - **Historical Baseline**: 1991-2020 climatological normals (WMO standard)                                      │
│ - **Recent Trends**: 2000-2023 for change detection                                                             │
│ - **Climate Projections**: Include only if specificall

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'initialize_subagent' with arguments: {'subtask_id': 'geo-framework', 'subtask_title':            │
│ 'Geographic Scope and Spatial Framework Definition', 'subtask_description': 'Define the exact spatial           │
│ boundaries of the study area per the research brief: Hauts-de-France, Normandy, northern Grand Est (Ardennes,   │
│ Meuse, Meurthe-et-Moselle), and Île-de-France. Identify and map three distinct zones: (1) Coastal zones along   │
│ the Channel/North Sea from Dunkirk to Le Havre, (2) Inland areas including Artois, Flanders, and Picardy        │
│ plateaus, (3) Urban agglomerations for heat island analysis (Lille, Rouen, Strasbourg, Paris). Select the five  │
│ mandatory representative stations (Dunkirk, Lille, Rouen, Strasbourg, Metz) plus at least 10 additional         │
│ stations to ensure adequate spatial coverage. Document the latitude, longitude, elevation, and data             │
│ availability period for each station. Create a base map showing administrative boundaries, zones, and station   │
│ locations. Quantify the maritime-continental gradient distance from coast to furthest inland point. This        │
│ framework will enable consistent spatial analysis across all climate variables. Use Météo-France station        │
│ metadata and geographic information systems. State explicitly that the four-region definition is being used per │
│ the standard recommendation since no alternative was specified.'}                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '# Climate of Northern France: A Comprehensive          │
│ Analysis\n\n## Executive Summary\n\nNorthern France exhibits a complex maritime-to-continental climate          │
│ transition across relatively short distances, influenced by Atlantic Ocean proximity, topographic features, and │
│ increasing urbanization. The region spans the administrative regions of Hauts-de-France, Normandy, northern     │
│ Grand Est (Ardennes, Meuse, Meurthe-et-Moselle), and Île-de-France, covering approximately 124,000 km² with     │
│ distinct climatic zones from coastal moderation to inland continental conditions.\n\nKey climate                │
│ characteristics include mean annual temperatures ranging from 9.8°C to 11.8°C depending on location and urban   │
│ effects, annual precipitation of 650-1100mm with significant orographic enhancement in elevated regions, and    │
│ pronounced seasonal cycles modulated by the North Atlantic Oscillation. Climate change signals are robust, with │
│ significant warming of +0.28°C/decade since 1991, altered precipitation patterns showing winter increases and   │
│ summer decreases, and intensifying extreme weather events.\n\n## Geographic and Spatial Framework\n\n###        │
│ Administrative Boundaries and Study Area\nThe analysis covers northern France using the four-region definition  │
│ per standard recommendations, encompassing:\n\n- **Hauts-de-France**: Northernmost region bordering Belgium and │
│ English Channel\n- **Normandy**: Western coastal region spanning English Channel to Atlantic Ocean  \n-         │
│ **Northern Grand Est**: Ardennes, Meuse, and Meurthe-et-Moselle departments\n- **Île-de-France**: Central       │
│ region centered on Paris metropolitan area\n\n### Distinct Climatic Zones\n\n**Coastal Zone (0-50km from        │
│ coastline)**\n- Linear extent: Dunkirk (51°02\'N) to Le Havre (49°30\'N)\n- Characteristics: Maritime           │
│ moderation, higher humidity, uniform precipitation\n- Key areas: Opal Coast, Alabaster Coast, English Channel   │
│ shoreline\n\n**Inland Zone (50-150km from coastline)**  \n- Topography: Rolling plateaus, chalk formations,     │
│ river valleys\n- Characteristics: Continental influence, greater temperature extremes\n- Key areas: Artois      │
│ plateau, Flanders plains, Picardy plateau\n\n**Urban Heat Island Zone**\n- Major cities: Lille, Rouen,          │
│ Strasbourg, Paris metropolitan areas\n- Effects: Temperature enhancement +0.5-1.4°C, modified precipitation     │
│ patterns\n- Urban-rural gradients extending 10-15km beyond city centers\n\n### Representative Station           │
│ Network\nFifteen meteorological stations provide comprehensive spatial coverage, including the five mandatory   │
│ stations (Dunkirk, Lille, Rouen, Strasbourg, Metz) plus ten additional stations ensuring adequate geographic    │
│ distribution for robust climate analysis.\n\n## Temperature Regime and Urban Heat Islands\n\n### Spatial        │
│ Temperature Patterns\nNorthern France exhibits clear maritime-inland temperature gradients with annual mean     │
│ temperatures ranging from:\n\n- **Coastal areas**: 10.8°C (Dunkirk) with minimal annual range (12-15°C)\n-      │
│ **Transition zones**: 10.4-10.6°C (Lille, Rouen) with moderate ranges (16-18°C)  \n- **Inland areas**:          │
│ 10.1-10.2°C (Metz, Strasbourg) with continental ranges (20-22°C)\n\n### Seasonal Characteristics\n\n**Winter    │
│ Temperature Regime (December-February)**\n- Coastal mean: 4-6°C with 15-25 frost days annually\n- Inland mean:  │
│ 1-3°C with 45-65 frost days annually  \n- Extreme minimum records: -15.2°C (Strasbourg, 1985), -12.8°C (Lille,  │
│ 1985)\n\n**Summer Temperature Regime (June-August)**\n- Coastal mean: 17.8-18.1°C with 1-2 heat wave days       │
│ annually\n- Inland mean: 19.2-20.1°C with 3-5 heat wav

Observations: # Climate of Northern France: A Comprehensive Analysis

## Executive Summary

Northern France exhibits a complex maritime-to-continental climate transition across relatively short distances, 
influenced by Atlantic Ocean proximity, topographic features, and increasing urbanization. The region spans the 
administrative regions of Hauts-de-France, Normandy, northern Grand Est (Ardennes, Meuse, Meurthe-et-Moselle), and 
Île-de-France, covering approximately 124,000 km² with distinct climatic zones from coastal moderation to inland 
continental conditions.

Key climate characteristics include mean annual temperatures ranging from 9.8°C to 11.8°C depending on location and
urban effects, annual precipitation of 650-1100mm with significant orographic enhancement in elevated regions, and 
pronounced seasonal cycles modulated by the North Atlantic Oscillation. Climate change signals are robust, with 
significant warming of +0.28°C/decade since 1991, altered precipitation patterns showing winter increases and 
summer decreases, and intensifying extreme weather events.

## Geographic and Spatial Framework

### Administrative Boundaries and Study Area
The analysis covers northern France using the four-region definition per standard recommendations, encompassing:

- **Hauts-de-France**: Northernmost region bordering Belgium and English Channel
- **Normandy**: Western coastal region spanning English Channel to Atlantic Ocean  
- **Northern Grand Est**: Ardennes, Meuse, and Meurthe-et-Moselle departments
- **Île-de-France**: Central region centered on Paris metropolitan area

### Distinct Climatic Zones

**Coastal Zone (0-50km from coastline)**
- Linear extent: Dunkirk (51°02'N) to Le Havre (49°30'N)
- Characteristics: Maritime moderation, higher humidity, uniform precipitation
- Key areas: Opal Coast, Alabaster Coast, English Channel shoreline

**Inland Zone (50-150km from coastline)**  
- Topography: Rolling plateaus, chalk formations, river valleys
- Characteristics: Continental influence, greater temperature extremes
- Key areas: Artois plateau, Flanders plains, Picardy plateau

**Urban Heat Island Zone**
- Major cities: Lille, Rouen, Strasbourg, Paris metropolitan areas
- Effects: Temperature enhancement +0.5-1.4°C, modified precipitation patterns
- Urban-rural gradients extending 10-15km beyond city centers

### Representative Station Network
Fifteen meteorological stations provide comprehensive spatial coverage, including the five mandatory stations 
(Dunkirk, Lille, Rouen, Strasbourg, Metz) plus ten additional stations ensuring adequate geographic distribution 
for robust climate analysis.

## Temperature Regime and Urban Heat Islands

### Spatial Temperature Patterns
Northern France exhibits clear maritime-inland temperature gradients with annual mean temperatures ranging from:

- **Coastal areas**: 10.8°C (Dunkirk) with minimal annual range (12-15°C)
- **Transition zones**: 10.4-10.6°C (Lille, Rouen) with moderate ranges (16-18°C)  
- **Inland areas**: 10.1-10.2°C (Metz, Strasbourg) with continental ranges (20-22°C)

### Seasonal Characteristics

**Winter Temperature Regime (December-February)**
- Coastal mean: 4-6°C with 15-25 frost days annually
- Inland mean: 1-3°C with 45-65 frost days annually  
- Extreme minimum records: -15.2°C (Strasbourg, 1985), -12.8°C (Lille, 1985)

**Summer Temperature Regime (June-August)**
- Coastal mean: 17.8-18.1°C with 1-2 heat wave days annually
- Inland mean: 19.2-20.1°C with 3-5 heat wave days annually
- Extreme maximum records: 38.2°C (Lille, 2019), 39.4°C (Strasbourg, 2003)

### Urban Heat Island Analysis
Urban heat island (UHI) effects significantly modify local temperature patterns:

**Lille Metropolitan Area**
- UHI intensity: +0.6°C annually (+1.8°C winter, +0.9°C summer)
- Peak winter enhancement: +2.1°C during calm, clear nights
- Urban expansion trends: Stable UHI magnitude despite background warming

**Paris Urban Heat Island**
- UHI intensity: +1.1°C annually (+2.3°C win

Final answer: # Climate of Northern France: A Comprehensive Analysis

## Executive Summary

Northern France exhibits a complex maritime-to-continental climate transition across relatively short distances, 
influenced by Atlantic Ocean proximity, topographic features, and increasing urbanization. The region spans the 
administrative regions of Hauts-de-France, Normandy, northern Grand Est (Ardennes, Meuse, Meurthe-et-Moselle), and 
Île-de-France, covering approximately 124,000 km² with distinct climatic zones from coastal moderation to inland 
continental conditions.

Key climate characteristics include mean annual temperatures ranging from 9.8°C to 11.8°C depending on location and
urban effects, annual precipitation of 650-1100mm with significant orographic enhancement in elevated regions, and 
pronounced seasonal cycles modulated by the North Atlantic Oscillation. Climate change signals are robust, with 
significant warming of +0.28°C/decade since 1991, altered precipitation patterns showing winter increases and 
summer decreases, and intensifying extreme weather events.

## Geographic and Spatial Framework

### Administrative Boundaries and Study Area
The analysis covers northern France using the four-region definition per standard recommendations, encompassing:

- **Hauts-de-France**: Northernmost region bordering Belgium and English Channel
- **Normandy**: Western coastal region spanning English Channel to Atlantic Ocean  
- **Northern Grand Est**: Ardennes, Meuse, and Meurthe-et-Moselle departments
- **Île-de-France**: Central region centered on Paris metropolitan area

### Distinct Climatic Zones

**Coastal Zone (0-50km from coastline)**
- Linear extent: Dunkirk (51°02'N) to Le Havre (49°30'N)
- Characteristics: Maritime moderation, higher humidity, uniform precipitation
- Key areas: Opal Coast, Alabaster Coast, English Channel shoreline

**Inland Zone (50-150km from coastline)**  
- Topography: Rolling plateaus, chalk formations, river valleys
- Characteristics: Continental influence, greater temperature extremes
- Key areas: Artois plateau, Flanders plains, Picardy plateau

**Urban Heat Island Zone**
- Major cities: Lille, Rouen, Strasbourg, Paris metropolitan areas
- Effects: Temperature enhancement +0.5-1.4°C, modified precipitation patterns
- Urban-rural gradients extending 10-15km beyond city centers

### Representative Station Network
Fifteen meteorological stations provide comprehensive spatial coverage, including the five mandatory stations 
(Dunkirk, Lille, Rouen, Strasbourg, Metz) plus ten additional stations ensuring adequate geographic distribution 
for robust climate analysis.

## Temperature Regime and Urban Heat Islands

### Spatial Temperature Patterns
Northern France exhibits clear maritime-inland temperature gradients with annual mean temperatures ranging from:

- **Coastal areas**: 10.8°C (Dunkirk) with minimal annual range (12-15°C)
- **Transition zones**: 10.4-10.6°C (Lille, Rouen) with moderate ranges (16-18°C)  
- **Inland areas**: 10.1-10.2°C (Metz, Strasbourg) with continental ranges (20-22°C)

### Seasonal Characteristics

**Winter Temperature Regime (December-February)**
- Coastal mean: 4-6°C with 15-25 frost days annually
- Inland mean: 1-3°C with 45-65 frost days annually  
- Extreme minimum records: -15.2°C (Strasbourg, 1985), -12.8°C (Lille, 1985)

**Summer Temperature Regime (June-August)**
- Coastal mean: 17.8-18.1°C with 1-2 heat wave days annually
- Inland mean: 19.2-20.1°C with 3-5 heat wave days annually
- Extreme maximum records: 38.2°C (Lille, 2019), 39.4°C (Strasbourg, 2003)

### Urban Heat Island Analysis
Urban heat island (UHI) effects significantly modify local temperature patterns:

**Lille Metropolitan Area**
- UHI intensity: +0.6°C annually (+1.8°C winter, +0.9°C summer)
- Peak winter enhancement: +2.1°C during calm, clear nights
- Urban expansion trends: Stable UHI magnitude despite background warming

**Paris Urban Heat Island**
- UHI intensity: +1.1°C annually (+2.3°C win

[Step 9: Duration 46.74 seconds| Input tokens: 92,063 | Output tokens: 7,986]

In [ ]:
print(result)

# Climate of Northern France: A Comprehensive Analysis

## Executive Summary

Northern France exhibits a complex maritime-to-continental climate transition across relatively short distances, influenced by Atlantic Ocean proximity, topographic features, and increasing urbanization. The region spans the administrative regions of Hauts-de-France, Normandy, northern Grand Est (Ardennes, Meuse, Meurthe-et-Moselle), and Île-de-France, covering approximately 124,000 km² with distinct climatic zones from coastal moderation to inland continental conditions.

Key climate characteristics include mean annual temperatures ranging from 9.8°C to 11.8°C depending on location and urban effects, annual precipitation of 650-1100mm with significant orographic enhancement in elevated regions, and pronounced seasonal cycles modulated by the North Atlantic Oscillation. Climate change signals are robust, with significant warming of +0.28°C/decade since 1991, altered precipitation patterns showing winter incre